# How to manage conversation history

One of the most common use cases for persistence is to use it to keep track of conversation history. This is great - it makes it easy to continue conversations. As conversations get longer and longer, however, this conversation history can build up and take up more and more of the context window. This can often be undesirable as it leads to more expensive and longer calls to the LLM, and potentially ones that error. In order to prevent this from happening, you need to probably manage the conversation history.

Note: this guide focuses on how to do this in LangGraph, where you can fully customize how this is done. If you want a more off-the-shelf solution, you can look into functionality provided in LangChain:

- [How to filter messages](https://js.langchain.com/v0.2/docs/how_to/filter_messages/)
- [How to trim messages](https://js.langchain.com/v0.2/docs/how_to/trim_messages/)

## Setup

First, let's set up the packages we're going to want to use

```bash
yarn add langchain @langchain/anthropic
```

Next, we need to set API keys for Anthropic (the LLM we will use)

```bash
export ANTHROPIC_API_KEY=your_api_key
```

In [7]:
if (!process.env.ANTHROPIC_API_KEY) {
  throw new Error("Missing ANTHROPIC_API_KEY environment variable");
}

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_CALLBACKS_BACKGROUND=true
export LANGCHAIN_API_KEY=your_api_key
```

## Build the agent
Let's now build a simple ReAct style agent.

In [1]:
import { ChatAnthropic } from "@langchain/anthropic";
import { tool } from "@langchain/core/tools";
import { BaseMessage, AIMessage } from "@langchain/core/messages";
import { StateGraph, START, END } from "@langchain/langgraph";
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { MemorySaver } from "@langchain/langgraph";
import { z } from "zod";

interface MessagesState {
    messages: BaseMessage[];
}

const memory = new MemorySaver();

const searchTool = tool((_): string => {
    // This is a placeholder for the actual implementation
    // Don't let the LLM know this though 😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
}, {
    name: "search",
    description: "Call to surf the web.",
    schema: z.object({
        query: z.string()
    })
})


const tools = [searchTool]
const toolNode = new ToolNode<MessagesState>(tools)
const model = new ChatAnthropic({ model: "claude-3-haiku-20240307" })
const boundModel = model.bindTools(tools)

async function shouldContinue(state: MessagesState): Promise<"action" | typeof END> {
    const lastMessage = state.messages[state.messages.length - 1];
    // If there is no function call, then we finish
    if (lastMessage && !(lastMessage as AIMessage).tool_calls?.length) {
        return END;
    }
    // Otherwise if there is, we continue
    return "action";
}

// Define the function that calls the model
async function callModel(state: MessagesState) {
    const response = await model.invoke(state.messages);
    // We return an object, because this will get merged with the existing state
    return { messages: [response] };
}

// Define a new graph
const workflow = new StateGraph<MessagesState>({
    channels: {
        messages: {
            reducer: (a: any, b: any) => a.concat(b)
        },
    }
})
    // Define the two nodes we will cycle between
    .addNode("agent", callModel)
    .addNode("action", toolNode)
    // We now add a conditional edge
    .addConditionalEdges(
        // First, we define the start node. We use `agent`.
        // This means these are the edges taken after the `agent` node is called.
        "agent",
        // Next, we pass in the function that will determine which node is called next.
        shouldContinue
    )
    // We now add a normal edge from `action` to `agent`.
    // This means that after `action` is called, `agent` node is called next.
    .addEdge("action", "agent")
    // Set the entrypoint as `agent`
    // This means that this node is the first one called
    .addEdge(START, "agent");

// Finally, we compile it!
// This compiles it into a LangChain Runnable,
// meaning you can use it as you would any other runnable
const app = workflow.compile({
    checkpointer: memory,
});

In [2]:
import { HumanMessage } from "@langchain/core/messages";

const config = { configurable: { thread_id: "2"}, streamMode: "values" as const }

const inputMessage = new HumanMessage("hi! I'm bob");
for await (const event of await app.stream({
    messages: [inputMessage]
}, config)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (1) =================================`)
    console.log(recentMsg.content);
}

console.log("\n\n================================= END =================================\n\n")

const inputMessage2 = new HumanMessage("what's my name?");
for await (const event of await app.stream({
    messages: [inputMessage2]
}, config)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (2) =================================`)
    console.log(recentMsg.content);
}


================================ human Message (1) =================================
hi! I'm bob
================================ ai Message (1) =================================
It's nice to meet you, Bob! As an AI assistant, I'm here to help out however I can. Feel free to ask me anything, and I'll do my best to provide useful information or assistance. How can I help you today?


================================= END =================================


================================ human Message (2) =================================
what's my name?
================================ ai Message (2) =================================
You said your name is Bob, so your name is Bob.


## Filtering messages

The most straight-forward thing to do to prevent conversation history from blowing up is to filter the list of messages before they get passed to the LLM. This involves two parts: defining a function to filter messages, and then adding it to the graph. See the example below which defines a really simple `filterMessages` function and then uses it.

In [5]:
import { ChatAnthropic } from "@langchain/anthropic";
import { tool } from "@langchain/core/tools";
import { BaseMessage, AIMessage } from "@langchain/core/messages";
import { StateGraph, START, END } from "@langchain/langgraph";
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { MemorySaver } from "@langchain/langgraph";
import { z } from "zod";

interface MessagesState {
    messages: BaseMessage[];
}

const memory = new MemorySaver();

const searchTool = tool((_): string => {
    // This is a placeholder for the actual implementation
    // Don't let the LLM know this though 😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
}, {
    name: "search",
    description: "Call to surf the web.",
    schema: z.object({
        query: z.string()
    })
})


const tools = [searchTool]
const toolNode = new ToolNode<MessagesState>(tools)
const model = new ChatAnthropic({ model: "claude-3-haiku-20240307" })
const boundModel = model.bindTools(tools)


async function shouldContinue(state: MessagesState): Promise<"action" | typeof END> {
    const lastMessage = state.messages[state.messages.length - 1];
    // If there is no function call, then we finish
    if (lastMessage && !(lastMessage as AIMessage).tool_calls?.length) {
        return END;
    }
    // Otherwise if there is, we continue
    return "action";
}

const filterMessages = (messages: BaseMessage[]): BaseMessage[] => {
    // This is very simple helper function which only ever uses the last message
    return messages.slice(-1);
}


// Define the function that calls the model
async function callModel(state: MessagesState) {
    const response = await model.invoke(filterMessages(state.messages));
    // We return an object, because this will get merged with the existing state
    return { messages: [response] };
}


// Define a new graph
const workflow = new StateGraph<MessagesState>({
    channels: {
        messages: {
            reducer: (a: any, b: any) => a.concat(b)
        },
    }
})
    // Define the two nodes we will cycle between
    .addNode("agent", callModel)
    .addNode("action", toolNode)
    // We now add a conditional edge
    .addConditionalEdges(
        // First, we define the start node. We use `agent`.
        // This means these are the edges taken after the `agent` node is called.
        "agent",
        // Next, we pass in the function that will determine which node is called next.
        shouldContinue
    )
    // We now add a normal edge from `action` to `agent`.
    // This means that after `action` is called, `agent` node is called next.
    .addEdge("action", "agent")
    // Set the entrypoint as `agent`
    // This means that this node is the first one called
    .addEdge(START, "agent");

// Finally, we compile it!
// This compiles it into a LangChain Runnable,
// meaning you can use it as you would any other runnable
const app = workflow.compile({
    checkpointer: memory,
});

In [6]:
import { HumanMessage } from "@langchain/core/messages";

const config = { configurable: { thread_id: "2"}, streamMode: "values" as const }

const inputMessage = new HumanMessage("hi! I'm bob");
for await (const event of await app.stream({
    messages: [inputMessage]
}, config)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (1) =================================`)
    console.log(recentMsg.content);
}

console.log("\n\n================================= END =================================\n\n")

const inputMessage2 = new HumanMessage("what's my name?");
for await (const event of await app.stream({
    messages: [inputMessage2]
}, config)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (2) =================================`)
    console.log(recentMsg.content);
}

================================ human Message (1) =================================
hi! I'm bob
================================ ai Message (1) =================================
Hello Bob! It's nice to meet you. As an AI assistant, I'm here to help with any questions or tasks you might have. Please feel free to ask me anything, and I'll do my best to assist you.


================================= END =================================


================================ human Message (2) =================================
what's my name?
================================ ai Message (2) =================================
I'm afraid I don't actually know your name. As an AI assistant, I don't have personal information about you unless you choose to provide it to me. Could you please tell me your name?


In the above example we defined the `filter_messages` function ourselves. We also provide off-the-shelf ways to trim and filter messages in LangChain. 

- [How to filter messages](https://js.langchain.com/v0.2/docs/how_to/filter_messages/)
- [How to trim messages](https://js.langchain.com/v0.2/docs/how_to/trim_messages/)